In [1]:
import os
import pandas as pd
import psycopg2

In [2]:
connection = psycopg2.connect(database="postgres", host="localhost", user="deveshsarda", password="macrostrat", port="5432")
cursor = connection.cursor()

In [3]:
cursor.execute("SET search_path TO macrostrat;")

In [4]:
def execute_query(query):
    try:
        cursor.execute(query)
        col_names = [desc[0] for desc in cursor.description]
        connection.commit()
        return pd.DataFrame(cursor.fetchall(), columns = col_names)
    except Exception as e:
        print("Got exception of", e)
        connection.rollback()
        return None

In [5]:
all_columns = execute_query(""" 
SELECT *
FROM information_schema.columns
WHERE table_schema = 'macrostrat'
""")
all_columns.head()

,table_catalog,table_schema,table_name,column_name,ordinal_position,column_default,is_nullable,data_type,character_maximum_length,character_octet_length,...,is_identity,identity_generation,identity_start,identity_increment,identity_maximum,identity_minimum,identity_cycle,is_generated,generation_expression,is_updatable
0,postgres,macrostrat,units_sections,col_id,4,None,NO,integer,NaN,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES
1,postgres,macrostrat,lookup_units,t_plat,9,None,YES,numeric,NaN,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES
2,postgres,macrostrat,lookup_units,t_plng,10,None,YES,numeric,NaN,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES
3,postgres,macrostrat,lookup_units,b_int,11,None,YES,integer,NaN,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES
4,postgres,macrostrat,lookup_units,b_int_age,13,None,YES,numeric,NaN,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES


In [6]:
all_columns = all_columns[["table_name", "column_name", "data_type"]]
all_columns = all_columns.sort_values(by = ["table_name"])
all_columns.to_csv("all_columns.csv", index = False)

In [7]:
unit_decription_df = execute_query(""" 
SELECT *
FROM units u
JOIN unit_liths ul
  ON u.id = ul.unit_id
JOIN liths l
  ON l.id = ul.lith_id
JOIN unit_lith_atts ula
  ON ula.unit_lith_id = ul.id
JOIN lith_atts la
  ON ula.lith_att_id = la.id
""")
unit_decription_df.head()

,id,strat_name,color,outcrop,fo,fo_h,lo,lo_h,position_bottom,position_top,...,lith_color,id,unit_lith_id,lith_att_id,ref_id,date_mod,id,lith_att,att_type,lith_att_fill
0,18662,Cape Rouge,yellow,,145,2,145,0,1.000,1.000,...,#FFD500,33909,515,45,0,2020-04-17 09:55:54,45,fine,grains,0
1,18819,Rocky Brook,gray,,143,2,142,3,1.000,1.000,...,#86919D,31264,776,80,0,2019-12-03 20:17:26,80,calcareous,lithology,0
2,18819,Rocky Brook,gray,,143,2,142,3,1.000,1.000,...,#86919D,31261,776,112,0,2019-12-03 20:15:57,112,red,color,0
3,18819,Rocky Brook,gray,,143,2,142,3,1.000,1.000,...,#4C71F1,31257,778,82,0,2019-12-03 20:14:39,82,dolomitic,lithology,0
4,18853,Fourchu,green,,203,2,203,3,2.000,2.000,...,#9E6244,2513,853,127,0,None,127,chloritic,lithology,0


In [8]:
unit_decription_df.columns

Index(['id', 'strat_name', 'color', 'outcrop', 'fo', 'fo_h', 'lo', 'lo_h',
       'position_bottom', 'position_top', 'max_thick', 'min_thick',
       'section_id', 'col_id', 'id', 'lith_id', 'unit_id', 'prop', 'dom',
       'comp_prop', 'mod_prop', 'toc', 'ref_id', 'date_mod', 'id', 'lith',
       'lith_group', 'lith_type', 'lith_class', 'lith_equiv', 'lith_fill',
       'comp_coef', 'initial_porosity', 'bulk_density', 'lith_color', 'id',
       'unit_lith_id', 'lith_att_id', 'ref_id', 'date_mod', 'id', 'lith_att',
       'att_type', 'lith_att_fill'],
      dtype='object')

In [9]:
filtered_units_df = unit_decription_df[["strat_name", "lith_id", "lith", "lith_group", "lith_type", "lith_class", "lith_att_id", "lith_att", "att_type"]]
filtered_units_df.head()

,strat_name,lith_id,lith,lith_group,lith_type,lith_class,lith_att_id,lith_att,att_type
0,Cape Rouge,10,sandstone,sandstones,siliciclastic,sedimentary,45,fine,grains
1,Rocky Brook,9,siltstone,mudrocks,siliciclastic,sedimentary,80,calcareous,lithology
2,Rocky Brook,9,siltstone,mudrocks,siliciclastic,sedimentary,112,red,color
3,Rocky Brook,30,limestone,None,carbonate,sedimentary,82,dolomitic,lithology
4,Fourchu,84,schist,None,metamorphic,metamorphic,127,chloritic,lithology


In [10]:
edges_to_extract = [("strat_name", "lith"), ("lith", "lith_group"), ("lith", "lith_type"), ("lith", "lith_group"), ("lith", "lith_att", "att_type")]
def extract_edges(row, output):
    for edge_details in edges_to_extract:
        edge_src = edge_details[0]
        edge_dst = edge_details[1]
        edge_name = edge_src + "_to_" + edge_dst
        if len(edge_details) == 3:
            edge_name = "att_" + row[edge_details[2]].replace(" ", "_")
        output.append([edge_name, row[edge_src], row[edge_dst]])

In [11]:
output_rows = []
for idx, row in filtered_units_df.iterrows():
    extract_edges(row, output_rows)

df = pd.DataFrame(output_rows, columns = ["edge_name", "edge_src", "edge_dst"])
df = df.drop_duplicates()
print(len(df.index))
df.head()

67424


,edge_name,edge_src,edge_dst
0,strat_name_to_lith,Cape Rouge,sandstone
1,lith_to_lith_group,sandstone,sandstones
2,lith_to_lith_type,sandstone,siliciclastic
4,att_grains,sandstone,fine
5,strat_name_to_lith,Rocky Brook,siltstone


In [12]:
df["edge_name"].value_counts()

edge_name
strat_name_to_lith    65228
att_lithology           701
att_color               587
att_grains              367
att_sed_structure       214
lith_to_lith_group      112
lith_to_lith_type       112
att_bedform              78
att_structure            25
Name: count, dtype: int64

In [13]:
df.to_csv("macrostrat_graph.csv", index = False)